# Generate collaboration metrics

In [29]:
import pandas as pd
from pathlib import Path, PurePath
import os
import sys
import numpy as np

In [3]:
from github2pandas_manager.config_parser import YAML_RequestDefinition
import aux

In [4]:
sys.path.append(os.path.join(sys.path[0],'..','..','FeedbackGeneration'))
from DataHandler import DataHandler

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
result_folder = "./example/notebooks/"
result_file_name = "F_CollaborationRatio"

In [14]:
project_folder_name = "../"
project_parameter_file = "config.yml"

In [15]:
# Just for local testing purposes 
# project_folder_name = "./example/"

In [16]:
request_params = YAML_RequestDefinition(project_folder_name+project_parameter_file)
print(request_params.parameters.project_folder)

./example/data/TaskExamples


In [17]:
path = PurePath(os.getcwd())
if path.name == "notebooks":
    print(os.getcwd())
    os.chdir("../../")
    print(os.getcwd())

In [18]:
dh = DataHandler(request_params.parameters.project_folder)

example/data/TaskExamples


In [19]:
dh.raw_pdCommits['team'] =  dh.raw_pdCommits['repo_name'].apply(aux.getTeamName)
dh.raw_pdEdits['team'] =  dh.raw_pdEdits['repo_name'].apply(aux.getTeamName)
dh.raw_pdIssues['team'] =  dh.raw_pdIssues['repo_name'].apply(aux.getTeamName)
dh.raw_pdRepositories['team'] =  dh.raw_pdRepositories['repo_name'].apply(aux.getTeamName)
dh.raw_pdUsers['team'] =  dh.raw_pdUsers['repo_name'].apply(aux.getTeamName)

In [20]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow", "github-actions[bot]", "GitHubAction"]
github_users_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 

tutor_black_list = ["SebastianZug", "andre-dietrich", "JayTee42", "galinarudollf", "bpaulwitz",
                    "Sebastian Zug", "gjaeger1", "Florian Richter", "RiFlorian", "JayTee"]
tutor_black_list_ids = dh.raw_pdUsers [dh.raw_pdUsers.login.isin(tutor_black_list)].anonym_uuid.unique().tolist() 

black_list_ids = tutor_black_list_ids + github_users_black_list_ids
black_list_ids

['open-young-friend-night',
 'take-certain-word-end',
 'happen-democratic-early-game',
 'expect-bring-medical-lot',
 'serve-stay-cultural-reason',
 'would-hot-power-president',
 'can-expect-late-month']

In [21]:
dh.raw_pdUsers['role'] = "student"
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids), "role"] = 'github'
dh.raw_pdUsers.loc[dh.raw_pdUsers.anonym_uuid.isin(tutor_black_list_ids), "role"] = 'tutor'

In [22]:
df = pd.merge(dh.raw_pdEdits, dh.raw_pdCommits[['commit_sha', 'author']], how='left', left_on=['commit_sha'], right_on = ['commit_sha'])
df = pd.merge(df, dh.raw_pdUsers[['anonym_uuid', 'role']], how='left', left_on=['author'], right_on = ['anonym_uuid'])

In [23]:
df_files = df[df.role != "github"].groupby(['team', 'author']).filename.unique().to_frame()
df_files_separated = pd.DataFrame(df_files.to_records(),
                                  columns=df_files.index.names + list(df_files.columns))

teams = df_files_separated.groupby('team').author.unique()
dict_teams =teams.to_dict()
print('open-yong-friend-night'  in dict_teams['andre_und_sebastian1'])

False


In [27]:
df_files = df[df.role != "github"].groupby(['team', 'author']).filename.unique().to_frame()
df_files_separated = pd.DataFrame(df_files.to_records(),
                                  columns=df_files.index.names + list(df_files.columns))

count = 0
authors_list =  ['author_A', 'author_B']
df['authorID'] = "Orga"
for index, row in df[df.role != "github"].iterrows():
    idx = list(dict_teams[row.team]).index(row.author)
    df.at[index, 'authorID'] = authors_list[idx]

pdResults = df.groupby(['team', 'authorID']).filename.unique().unstack()
pdResults

authorID,Orga,author_A,author_B
team,,,
andre_und_sebastian1,"[CSharpBasics.txt, README.md, team.config]",[Task.md],NaN
galina_und_jonas,"[CSharpBasics.txt, README.md, team.config]",[team.config],"[team.config, CSharpBasics.md, HelloWorld.cs]"
nico_und_baldur,"[CSharpBasics.txt, README.md, team.config]","[team.config, CSharpBasics.md, CSharpBasics.txt]",[team.config]


In [68]:
pdResults['CollaborationRatio'] = 0
for index, row in pdResults.iterrows():
    print("---------------------")
    if (not isinstance(row.author_A,float)):
        A_files = list(row.author_A)
    else:
        A_files = []
    if (not isinstance(row.author_B,float)):
        B_files = list(row.author_B)
    else:
        B_files = []
    
    if A_files and B_files:
        all_files = set(A_files) | set(B_files)
    if A_files and not B_files:
        all_files = set(A_files)
    if not A_files and B_files:
        all_files = set(B_files)

    pdResults.at[index, 'CollaborationRatio'] = len(set(A_files) & set(B_files)) / len(all_files)

---------------------
['Task.md']
[]
{'Task.md'}
---------------------
['team.config']
['team.config', 'CSharpBasics.md', 'HelloWorld.cs']
{'team.config', 'HelloWorld.cs', 'CSharpBasics.md'}
---------------------
['team.config', 'CSharpBasics.md', 'CSharpBasics.txt']
['team.config']
{'CSharpBasics.txt', 'team.config', 'CSharpBasics.md'}


In [69]:
pdResults.reset_index(inplace=True)
pdResults['team'] = "`" + pdResults['team'] + "`" 
pdResults

authorID,team,Orga,author_A,author_B,CollaborationRatio
0,`andre_und_sebastian1`,"[CSharpBasics.txt, README.md, team.config]",[Task.md],NaN,0.000000
1,`galina_und_jonas`,"[CSharpBasics.txt, README.md, team.config]",[team.config],"[team.config, CSharpBasics.md, HelloWorld.cs]",0.333333
2,`nico_und_baldur`,"[CSharpBasics.txt, README.md, team.config]","[team.config, CSharpBasics.md, CSharpBasics.txt]",[team.config],0.333333


In [26]:
with open(result_folder + result_file_name + '.txt', 'w') as f:
    f.write(pdResults[['team', 'Orga', 'author_A', 'author_B']].to_markdown())